In [1]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
from urllib.parse import urlparse, urljoin
import re

##dbs星展, sinopac永豐
bank_urls = {
            'dbs':'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/default.page', 
            'sinopac':'https://bank.sinopac.com/sinopacBT/personal/credit-card/introduction/list.html',
            'cathy': 'https://www.cathaybk.com.tw/cathaybk/personal/product/credit-card/cards/',
            'firstbank': 'https://card.firstbank.com.tw/sites/card/CreditCardList'
             }
credit_card_urls = {}
soup_documents = {k:None for k in list(bank_urls.keys())}

for bank, url in bank_urls.items():
    loader = RecursiveUrlLoader(url=url)
    docs = loader.load()
    if bank == 'dbs':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if (a['href'].startswith('/personal-zh/cards')) and (a['href'].endswith('hyperlink'))]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'sinopac':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if a['href'].startswith('./')]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'firstbank':
        def firstbank_extractor(html:str):
            soup_obj = Soup(html, 'html.parser')
            credit_card_feature = [re.sub(r'\n+', '\n', d.text) for d in soup_obj.find_all('div', {"class":[["card-single-features"]]})]
            txt = ''
            for features in  credit_card_feature:
                split = features.split('\n', 1)
                txt += f"信用卡卡名：{split[0].strip()}\n信用卡特色: {split[1]}"
            return txt
        all_html = RecursiveUrlLoader(bank_urls['firstbank'], extractor=firstbank_extractor).load()
        all_html[0].metadata['bank'] = 'firstbank'
        soup_documents[bank] = all_html

        divs = [Soup(doc.page_content, 'html.parser').find_all('div', {"class":'card-single'}) for doc in docs][0]
        links = []
        for d in divs:
            link_tags = d.find_all('a')
            if len(link_tags) == 1:
                detail_url = link_tags[0]['href']
                links.append(detail_url)
            else:
                for tag in link_tags:
                    if tag.text == '詳細內容':
                        links.append(tag['href'])
                    else:
                        continue
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'cathy':
        def parser(html):
            ## main page
            divs = Soup(html, 'html.parser').find_all('div', {'class':'cubre-m-compareCard -credit'})
            if divs:
                divs = [re.sub(r'\n+', '\n', i.text) for i in divs]
                txt = (' ').join(divs)
                txt = re.sub(r' +', ' ', txt)
                txt = [t.replace('\n立即申辦','', 1) for t in txt.split('詳細說明') if t.startswith('\n立即申辦')]
                txt = ('').join(txt).replace('\n \n', '\n')
            else:
                ## deeper page
                divs = Soup(html, 'html.parser').find_all('div', class_=["cubre-o-textContent", "cubre-m-colorBanner__title","cubre-m-iconEssay__title","cubre-m-horGraphic__title","cubre-m-remind__title","cubre-m-puzzle__title","cubre-a-kvTitle -card"])
                divs = [d.text for d in divs if '您將離開本行官網 前往外部網站' not in d.text]
                uni_divs = []
                for d in divs[:len(divs)-1]:
                    if d not in uni_divs:
                        uni_divs.append(d)
                txt = re.sub(r'\n+', '\n', ('\n').join(uni_divs))
                txt = re.sub(r' +', ' ', txt)
            return txt
            
        docs = RecursiveUrlLoader(url=url, extractor=parser).load()
        for d in docs:
            d.metadata['bank'] = '國泰銀行'
            d.metadata['credit card'] = d.metadata['title'].split('-', 1)[0].strip()
        docs = [d for d in docs if '停發' not in d.metadata['title']]
        soup_documents['cathy'] = docs
        

In [ ]:
from langchain_community.document_loaders.url_selenium import SeleniumURLLoader
# documents = {}
# for bank, links in credit_card_urls.items():
#     documents[bank] = SeleniumURLLoader(urls=links).load()
        

In [2]:
def sinopac_extractor(html: str) -> str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'tab-box'})]
    divs_txt.insert(0, re.sub(r'\n\n+', '\n', soup.find('div', {'class':'info'}).text))
    div_set = []
    for txt in divs_txt:
        if txt not in div_set:
            div_set.append(txt)
    txt = re.sub(r'[\t\r\xa0]', '', ('\n').join(div_set))
    txt = re.sub(r'  +', ' ', txt)
    txt = re.sub(r'\n+', '\n', txt)
    return txt.strip()

def dbs_extractor(html:str)->str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'flpweb-legacy'})]
    if divs_txt:
        div_set = []
        for txt in divs_txt:
            if txt not in div_set:
                div_set.append(txt)
        all_txt = ('\n').join(div_set)
    else:
        txt = soup.text
        txt = [t.strip() for t in txt.split('\n') if( t != 'more..') and (len(txt.strip()) != 1)]
        all_txt = ('\n').join(txt)

    all_txt = all_txt.replace('個人網路銀行\nCard+ 信用卡數位服務\n企業網路銀行', '')
    all_txt = re.sub(r'[\t\r\xa0]', '', all_txt)
    all_txt = re.sub(r'\xa0', '', all_txt)
    all_txt = re.sub(r'  +', ' ', all_txt)
    all_txt = re.sub(r'\n+', '\n', all_txt)
    all_txt = all_txt.split('信用卡刷卡優惠更多星展信用卡刷卡優惠信息，詳情請見刷卡優惠說明。')[0]

    if '假字假字假字假字假字假字假字假字假字假字' in all_txt:
        all_txt = re.sub('假字假字假字+', '', all_txt)

    pattern = r"網銀登入\n[\s\S]*?選擇您的網站 ▼\n個人金融\n個人金融\n財富管理\n星展豐盛理財\n星展豐盛私人客戶\n企業金融\n中小企業銀行\n企業及機構銀行\n星展集團\n關於我們"
    # Replace the matched portion with an empty string
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'Loading...\n尊享至上\n[\s\S]*?\n權益手冊\n更多獨享優惠'
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'新戶好禮\n[\s\S]*?\n更多星展信用卡'
    all_txt = re.sub(pattern, '', all_txt)
    all_txt = all_txt.split('謹慎理財 信用至上')[0]
    return all_txt.strip()

def firstbank_extractor(html:str)->str:
    divs = Soup(html, 'html.parser').find_all('div', {"class":['carousel-item', 'content-body']})
    return ('').join([re.sub(r'\n+', '\n', d.text) for d in divs])

for bank, links in credit_card_urls.items():
    docs = []
    if bank == 'sinopac':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=sinopac_extractor).load()
            doc[0].metadata['bank'] = '永豐銀行'
            docs.extend(doc)
    elif bank == 'dbs':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=dbs_extractor).load()
            if doc:
                doc[0].metadata['bank'] = '星展銀行'
                docs.extend(doc)
    elif bank == 'firstbank':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=firstbank_extractor).load()
            doc[0].metadata['bank'] = '第一銀行'
            docs.extend(doc)
    if soup_documents[bank]:
        soup_documents[bank].extend(docs)
    else:
        soup_documents[bank] = docs

In [3]:
for k,v in soup_documents.items():
    print(f"Bank {k}: {len(v)}")

Bank dbs: 23
Bank sinopac: 46
Bank cathy: 18
Bank firstbank: 37


## Extract credit card name from DBS, Sinopac

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import string
import re

chinese_punctuation = "。！；：，、【】《》『』——"
translator = str.maketrans('', '', chinese_punctuation)

for k in list(soup_documents.keys()):
    if (k == 'sinopac'):
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.page_content.split('\n')[0]
            # print(credit_card)
            soup_documents[k][index].metadata['credit card'] = credit_card
    elif k == 'dbs':
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.page_content.split('\n')[0].split('|')[0].translate(translator)
            match = re.search(r'星展.*?卡', credit_card)
            if match:
                soup_documents[k][index].metadata['credit card'] = match.group()  # Output: 星展HAPPY GO聯名卡
            else:
                match = re.search(r'星展.*?卡', doc.metadata['description'])
                if match:
                    soup_documents[k][index].metadata['credit card'] = match.group()
                else:
                    pass
    elif k == 'firstbank':
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.metadata['title'].strip()
            match = re.search(r'^(.*?)\s*-\s*信用卡', credit_card)
            if match:
                credit_card = match.group(1).split('_')[0]
                if credit_card.endswith('卡') or 'pay' in credit_card.lower():
                    soup_documents[k][index].metadata['credit card']  = credit_card
                else:
                    soup_documents[k][index].metadata['credit card']  = ''
            else:
                pass

In [9]:
import mlflow
from langchain.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=300,
                                        length_function=len,
                                        is_separator_regex=True,
                                        chunk_overlap=50,
                                        separators=['。'])
banks = list(bank_urls.keys())
chunks = splitter.split_documents(soup_documents[banks[0]])
for b in banks[1:]:
    chunks.extend(splitter.split_documents(soup_documents[b]))

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-base-zh-v1.5', model_kwargs={'device':'mps'})
collection_name = 'credit_card_chunk300'
qdrant_url = 'http://localhost:6333'
# vec_store = QdrantVectorStore.from_existing_collection(
#                               collection_name=collection_name,
#                               embedding = embedding,
#                               url = qdrant_url)

vec_store = QdrantVectorStore.from_documents(chunks,
                              collection_name=collection_name,
                              embedding = embedding,
                              force_recreate=True,
                              url = qdrant_url)



### Graph DB Neo4j

In [35]:
from langchain_community.vectorstores import Neo4jVector
username='neo4j'
password='password'
url = 'neo4j://localhost:7687'
neo4j_db = Neo4jVector.from_documents(
            chunks,
            url=url,
            username=username,
            password=password,
            embedding=embedding,
            index_name=collection_name,
            node_label="CreditCardChunk",
            pre_delete_collection=True,  # Delete existing PDF data
        )

In [52]:
docs

[Document(metadata={'source': 'https://card.firstbank.com.tw/sites/card/zh_TW/1565690685886', 'content_type': 'text/html; charset=UTF-8', 'title': 'GLORY+世界卡 - 信用卡 | 第一銀行', 'description': '一銀百年來與台灣社會、經濟一起成長、茁壯，一貫秉持「顧客至上，服務第一」的經營理念，爲客戶的財富與託付創造更多的價值。期許能成為活躍亞洲的區域型銀行；予客戶最滿意之金融服務，予股東最豐碩且穩健之獲利，予員工最佳的生涯發展空間。', 'language': 'zh-hant', 'bank': '第一銀行', 'credit card': 'GLORY+世界卡'}, page_content='\nGLORY+世界卡\n\n尊榮禮遇 榮耀非凡\n外幣消費享2.2%回饋\n保費最高享1.6%回饋\n高爾夫球場擊球優惠\nEZTABLE最高享20％現折優惠\n國際組織飯店住宿優惠\n\n\n GLORY+世界卡外幣消費享2.2%回饋\n出國消費 外幣消費享2.2%回饋\n 世界卡保費最高享1.6%回饋\n每戶每月回饋上限5,000元\n 世界卡高爾夫享擊球優惠價\nGLORY+世界卡及義享世界卡皆適用\n 世界卡EZTABLE最高現折20%優惠\nGLORY+世界卡及義享世界卡皆適用\n 世界卡agoda訂房最優享88折優惠\n刷一銀世界卡享專屬優惠\n 世界卡精選餐飲優惠 65折起\nGLORY+世界卡及義享世界卡皆適用\n 世界卡頂級飯店平日住宿 | 泡湯優惠\nGLORY+世界卡及義享世界卡皆適用\n 義大皇家酒店 全館餐廳尊享85折\n精選餐廳刷一銀卡享85折\n 義大天悅飯店\xa0全館餐廳尊享9折\n刷一銀卡享9折\n 【瘋旅遊】\n分期零利率 最高享回饋8,500元！\n 可樂旅遊 X 暢遊美加\n刷一銀卡，兩人成行現折6,000元\r\n刷世界卡，指定行程兩人折8,000元\n 世界卡全球頂級酒店預訂服務\n刷一銀世界卡享優惠\n 世界卡Club Med 預訂優惠\n刷一銀世界卡享優惠\n 世界卡旅遊保險服務\n 搭高鐵享回饋\n筆筆享5%回饋！\n GLORY+世界卡紅利酬賓\n外幣

## Self-querying retriever

In [60]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="bank",
        description="The bank that hosts the credit card webpage",
        type="string"
    ),
    AttributeInfo(
        name="credit card",
        description="The credit card name",
        type="string or list[string]"
    ),
]
document_content_description = "The texts between HTML tags that is displayed on a credit card webpage, such as headings, paragraphs, links, and other textual elements"
llm = ChatOllama(model='qwen2:7b-instruct', temperature=0, format='json')
self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vec_store,
    document_content_description,
    metadata_field_info,
    search_kwargs={'k': 7}
)

In [72]:
query = "CUBE 簽帳金融卡什麼通路有優惠？"
self_query_retriever.invoke(query)


In [ ]:

docs_with_score = neo4j_db.similarity_search_with_score(query, k=5)
docs_with_score

[(Document(metadata={'title': 'iLEO VISA 金融卡 - 信用卡 | 第一銀行', 'source': 'https://card.firstbank.com.tw/sites/card/zh_TW/1565690699316', 'credit card': 'iLEO VISA 金融卡', 'description': '一銀百年來與台灣社會、經濟一起成長、茁壯，一貫秉持「顧客至上，服務第一」的經營理念，爲客戶的財富與託付創造更多的價值。期許能成為活躍亞洲的區域型銀行；予客戶最滿意之金融服務，予股東最豐碩且穩健之獲利，予員工最佳的生涯發展空間。', 'bank': '第一銀行', 'language': 'zh-hant', 'content_type': 'text/html; charset=UTF-8'}, page_content='\niLEO VISA金融卡\n\n兼具金融卡及簽帳卡功能，簽帳享0.5%現金回饋\n兼具金融卡/簽帳卡功能\n數位申辦免出門\n簽帳0.5%現金回饋\n跨行提款及跨行轉帳手續費減免\n\n\n 簽帳0.5%現金回饋\n 滿額減免3次跨行提款手續費\n 【瘋旅遊】\n分期零利率 最高享回饋8,500元！\n 【瘋旅宿】\n滿額最高回饋1,000元！\n 【瘋訂房】\n滿額最高回饋1,000元！\n 寶雅天天滿額贈活動\n每月最高回饋10,000點\n 屈臣氏天天滿額贈\n單筆滿888元且登錄，享100元刷卡金回饋\n 康是美週五滿額贈\n門市+網購 滿額享回饋\n 美妝通路滿額享回饋-杏一\n實體門市單筆滿1,000元贈88元刷卡金\n 全國加油站\n筆筆享優惠\n 台亞加油站\n筆筆享優惠\n 刷卡零風險 聰明安心購 \n 卡片功能/用卡須知\n 常見問答\n\n\n申辦條件\n只要年滿15歲之本國籍自然人於本行開立數位綜合管理帳戶，即可申辦iLEO VISA金融卡，惟需成年始可於國外簽帳消費及提款'),
  0.8338145017623901),
 (Document(metadata={'title': 'iLEO信用卡 - 信用卡 | 第一銀行', 'source': 'https://card.firstbank.com.tw/sites/card/zh_TW/15656906

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder 
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

retriever = vec_store.as_retriever(search_kwargs={'k':10, 'score_threshold':0.5})
    
llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

reranker = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-base')
compressor = CrossEncoderReranker(model=reranker, top_n=5)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)


In [ ]:
from langchain.retrievers import EnsembleRetriever

try:
    ensemble_retriever = EnsembleRetriever(retrievers=[self_query_retriever, compression_retriever], weights=[0.6, 0.4])
    retrieved_docs = ensemble_retriever.invoke(query)
except:
    ensemble_retriever = compression_retriever
    retrieved_docs = ensemble_retriever.invoke(query)

retrieved_docs

[Document(metadata={'source': 'https://card.firstbank.com.tw/sites/card/zh_TW/1565690686302', 'content_type': 'text/html; charset=UTF-8', 'title': 'Wonderful星璨卡 - 信用卡 | 第一銀行', 'description': '一銀百年來與台灣社會、經濟一起成長、茁壯，一貫秉持「顧客至上，服務第一」的經營理念，爲客戶的財富與託付創造更多的價值。期許能成為活躍亞洲的區域型銀行；予客戶最滿意之金融服務，予股東最豐碩且穩健之獲利，予員工最佳的生涯發展空間。', 'language': 'zh-hant', 'bank': '第一銀行', 'credit card': 'Wonderful星璨卡', '_id': '3f9c40b7-95af-4f2e-b1ab-4c1ad154d5b3', '_collection_name': 'credit_card_chunk300'}, page_content='。本活動併入「國內外指定網購」活動名額。\n\xa0\n【同場加映】指定信用卡最高再享2%現金回饋\n卡片名稱\n再享加碼回饋\n活動內容\niLEO信用卡\n2%\n(限單筆消費)\n原卡片權益0.5%回饋無上限；\r\n\t\t\t活動加碼1.5%，每期帳單回饋上限500元\r\n\t\t\t(iLEO信用卡3期0利率自動分期無回饋)\nWonderful星璨卡\n1.5%\n(限單筆消費)\n原卡片權益0.5%回饋無上限；\r\n\t\t\t活動加碼1%，每期帳單回饋上限500元\n※回饋僅適用單筆一次付清，回饋排除聯合信用卡中心商店分期交易。\n注意事項\n本活動排除公司卡與簽帳金融卡'),
 Document(metadata={'source': 'https://card.firstbank.com.tw/sites/card/zh_TW/1565690699316', 'content_type': 'text/html; charset=UTF-8', 'title': 'iLEO VISA 金融卡 - 信用卡 | 第一銀行', 'description': '一銀百年來與台灣社會、經濟一起成長、茁

In [ ]:

from langchain_core.runnables import RunnablePassthrough
import opencc

from opik.integrations.langchain import OpikTracer
import os
import opik
# opik.configure(use_local=True)

# Create the Opik tracer
# opik_tracer = OpikTracer(tags=["langchain", "ollama"])
# os.environ["OPIK_PROJECT_NAME"] = "credit-card-helper"
converter = opencc.OpenCC('s2t.json')


model = ChatOllama(model='qwen2:7b-instruct', temperature=0)
# .with_config({"callbacks": [opik_tracer]})

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI question-answering assistant. Your task is to answer the question based on the provided documents. The documents are part of the text from the description of the credit cards.
    The documents are not all relevant to the question. Please filter and reply with an answer. No pre-amble or explanation. Please output text only, not in markdown format.
    
    Documents: 
    {context}

    Question:
    {question}

    Answer:""",
)

def format_docs(docs):
    return "\n\n".join([f"DOCUMENT {index}\nTitle: {d.metadata['title']}\nReleased bank: {d.metadata['bank']}\n{d.page_content}" for index, d in enumerate(docs)])


chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | model
    | StrOutputParser()
)

# question = "國泰銀行有什麼信用卡？"
response = chain.invoke(query)
# compression_retriever.invoke(question)
print(converter.convert(response))

iLEO信用卡和iLEO VISA金融卡的主要區別在於功能和服務。iLEO VISA金融卡具有金融卡及籤賬卡的功能，提供數字申請免出門、籤賬0.5%現金回饋、跨行提款及跨行轉賬手續費減免等服務。而iLEO信用卡則提供了海外消費最高9%回饋、指定行動支付最高4%回饋、新戶最高回饋3,600元、分期再享3期0利率等功能，並且支持綁定行動支付以獲得額外的回饋，如臺灣Pay、LINE Pay、街口支付等。此外，iLEO信用卡還提供外幣及特定通路2%現金回饋、旅行社和指定網購2%回饋、機場貴賓室服務等特色服務。因此，兩者在功能、回饋政策和服務上都有所不同。
